In [2]:
# Translated to .py by Meritxell Pacheco (December 2016)
# Adapted to PandasBiogeme by Nicola Ortelli (November 2019)


import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta
from biogeme.models import loglogit
from sklearn.model_selection import train_test_split

df = pd.read_csv("Data/swissmetro.dat", sep = '\t')
database = db.Database("swissmetro", df)

globals().update(database.variables)

# Exclude data
exclude = (( PURPOSE != 1 ) * ( PURPOSE != 3 ) + ( CHOICE == 0 )) > 0
database.remove(exclude)

df_train, df_test = train_test_split(df, test_size=0.2, random_state = 42)

database_train = db.Database("swissmetro_train", df_train)
database_test = db.Database("swissmetro_test", df_test)

# Parameters to be estimated
ASC_CAR   = Beta('ASC_CAR', 0, None, None, 0)
ASC_SM    = Beta('ASC_SM',  0, None, None, 0)
ASC_TRAIN = Beta('ASC_SBB', 0, None, None, 1)

B_TIME = Beta('B_TIME', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)
B_HE   = Beta('B_HE',   0, None, None, 0)

# Definition of new variables
TRAIN_COST = database_train.DefineVariable('TRAIN_COST', TRAIN_CO * ( GA == 0 ))
SM_COST    = database_train.DefineVariable('SM_COST', SM_CO * ( GA == 0 ))

# Utilities
V_TRAIN = ASC_TRAIN + B_TIME * TRAIN_TT + B_COST * TRAIN_COST + B_HE * TRAIN_HE
V_SM    = ASC_SM    + B_TIME * SM_TT    + B_COST * SM_COST    + B_HE * SM_HE
V_CAR   = ASC_CAR   + B_TIME * CAR_TT   + B_COST * CAR_CO

V = {1: V_TRAIN, 2: V_SM, 3: V_CAR}
av = {1: TRAIN_AV, 2: SM_AV, 3: CAR_AV}

logprob = loglogit(V, av, CHOICE)
biogeme = bio.BIOGEME(database_train, logprob)
biogeme.modelName = "Base Model"

In [27]:
V_TRAIN.getElementaryExpression('GROUP')

In [23]:
database.variables

{'GROUP': GROUP,
 'SURVEY': SURVEY,
 'SP': SP,
 'ID': ID,
 'PURPOSE': PURPOSE,
 'FIRST': FIRST,
 'TICKET': TICKET,
 'WHO': WHO,
 'LUGGAGE': LUGGAGE,
 'AGE': AGE,
 'MALE': MALE,
 'INCOME': INCOME,
 'GA': GA,
 'ORIGIN': ORIGIN,
 'DEST': DEST,
 'TRAIN_AV': TRAIN_AV,
 'CAR_AV': CAR_AV,
 'SM_AV': SM_AV,
 'TRAIN_TT': TRAIN_TT,
 'TRAIN_CO': TRAIN_CO,
 'TRAIN_HE': TRAIN_HE,
 'SM_TT': SM_TT,
 'SM_CO': SM_CO,
 'SM_HE': SM_HE,
 'SM_SEATS': SM_SEATS,
 'CAR_TT': CAR_TT,
 'CAR_CO': CAR_CO,
 'CHOICE': CHOICE,
 '__bioRemove__': __bioRemove__}

In [32]:
V_TRAIN

'Times'

In [22]:
for k, v in V.items():
    print(v.get_children())

[((ASC_SBB(fixed=0) + (B_TIME(init=0) * TRAIN_TT)) + (B_COST(init=0) * TRAIN_COST)), (B_HE(init=0) * TRAIN_HE)]
[((ASC_SM(init=0) + (B_TIME(init=0) * SM_TT)) + (B_COST(init=0) * SM_COST)), (B_HE(init=0) * SM_HE)]
[(ASC_CAR(init=0) + (B_TIME(init=0) * CAR_TT)), (B_COST(init=0) * CAR_CO)]


In [7]:
V[1].left.left.right.left

biogeme.expressions.Beta

In [8]:
biogeme.getBoundsOnBeta('B_TIME')

(None, None)